In [ ]:
#!pip install git+https://github.com/hmmlearn/hmmlearn

In [ ]:
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt

In [ ]:
# hidden states (temeperature)
hidden_states = {'cold': 0,
                 'hot': 1
                }
# observable states (numbers of eaten cones)
observable_states = [0, 1, 2]

## Likelihood - the forward algorithm

In this case, we're working with a pre trained HMM - which means all the essential parameters must be set manually.

In [ ]:
model = hmm.MultinomialHMM(n_components = len(hidden_states), verbose=True)

# initial probability vector of length n_hidden_states
model.startprob_ = np.array([0.5, 0.5])

# transition matrix - likelihood of transitioning between hidden states
# shape = n_hidden_states x n_hidden_states
model.transmat_ = np.array([
  [0.7, 0.3],
  [0.4, 0.6]
])

# emission matrix - likelihood of particular observation given each hidden state
# shape = n_hidden_states x n_observable_states
model.emissionprob_ = np.array([
  [0.7, 0.3, 0.0],
  [0.0, 0.4, 0.6]
])

In [ ]:
observations = np.atleast_2d([0, 2, 2]).T
np.e ** model.score(observations)

## Most likely sequence - Viterbi algorithm

In [ ]:
logprob, states = model.decode(observations)
[list(hidden_states.keys())[s] for s in states]

## Training a HMM - forward-backward algorithm

In [ ]:
def categorical_distribution(center, distr_range, n_categories):
    assert(center in range(n_categories))
    assert(distr_range < n_categories)
    broad_result = np.zeros(2*n_categories)
    distr_range += 1
    for i in range(distr_range):
        broad_result[n_categories + i] = broad_result[n_categories - i] = distr_range - i
        
    left = n_categories - center
    right = 2*n_categories - center
    result = broad_result[left:right]
    return result / result.sum() # normalization, so that sum =1

In order to generate data, let's generate probabilities of transistions between observable states.

In [ ]:
obs_trans_probs = np.array(
    [categorical_distribution(i, 2, len(observable_states)) for i in observable_states]
)

obs_initial_probs = np.ones(len(observable_states)) / len(observable_states)
print('initial')
print(obs_initial_probs)
print('transitions')
print(obs_trans_probs)

In [ ]:
def generate_sequence(length, init_probs=obs_initial_probs, trans_probs=obs_trans_probs, n_categories=len(observable_states)):
    result = []
    result.append(np.random.choice(n_categories, p=init_probs))
    for i in range(length -1):
        result.append(np.random.choice(n_categories, p=obs_trans_probs[result[i]]))
    
    presence = [i in result for i in range(n_categories)]
    return result if all(presence) else generate_sequence(length, init_probs, trans_probs, n_categories)

generate_sequence(10)

In [ ]:
data_train = np.array([generate_sequence(10) for _ in range(100)])
data_train

In [ ]:
model = hmm.MultinomialHMM(n_components=len(hidden_states))

model = model.fit(data_train) #, lengths=[len(d) for d in data_train])

print('startprob')
print(model.startprob_)
print('transmat')
print(model.transmat_)
print('emissions')
print(model.emissionprob_)

In [ ]:
Z2 = remodel.predict(X)